Code using API call

In [ ]:
!pip install transformers diffusers huggingface_hub langchain openai langchain_together


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.37
    Uninstalling langchain-core-0.3.37:
      Successfully uninstalled langchain-core-0.3.37


In [ ]:
# API Keys
TOGETHER_API_KEY = ""
HUGGING_FACE_API_KEY = ""

print("API keys have been entered successfully.")


API keys have been entered successfully.


In [ ]:
from langchain_together import ChatTogether, Together
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Initialize ChatTogether and model choice
loc_llm = ChatTogether(
    together_api_key=TOGETHER_API_KEY,
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    max_tokens=500,
    temperature=0.1
)

# Define a prompt template for recognizing entities in address
loc_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert in recognizing address components. I will provide you with a text containing an address in Persian, and you should identify and separate its components using commas. The components may include parts like 'Boulevard', 'Street', 'Alley', 'Building Number', etc. Ensure related terms are kept together. Be cautious with names and always group them appropriately. Here are some examples:"),
    ("system", "'بلوار فردوس رامین جنوبی دهم پلاک 7' should be 'بلوار فردوس, رامین جنوبی, دهم, پلاک 7'."),
    ("system", "'میدان هفت تیر ابراهیمی حسینی پلاک 5' should be ' 5 میدان هفت تیر، ابراهیمی، حسینی، پلاک '."),
    ("system", "'فلکه دوم صادقیه بزرگراه آیت الله کاشانی خیابان عقیل کوچه دهم پلاک 8' should be 'فلکه دوم صادقیه, بزرگراه آیت الله کاشانی, خیابان عقیل, کوچه دهم, پلاک 8'."),
    ("user", "Address: {address}")
])

# Create an LLMChain instance
loc_chain = LLMChain(llm=loc_llm, prompt=loc_template)

# Function to process address
def process_address(address):
    result = loc_chain.invoke({"address": address})
    return result['text']

# Example addresses for testing
persian_addresses = [
    "میدان سینا بزرگمهر احمدی پلاک 4",
    "لواسان استخر بوستان جنگلی یاس",
    "شهرک شهید الماسی بلوار هجرت انتهای هجرت",
    "سرخه حصار بزرگراه یاسینی بوستان جنگلی سرخه حصار",
    "دماوند بلوار اتحاد قبل از اول شرقی آکس چوب"
]

# Process and print each address
for address in persian_addresses:
    processed_address = process_address(address)
    print(f"Original Address: {address}")
    print(f"Processed Address: {processed_address}\n")


Original Address: میدان سینا بزرگمهر احمدی پلاک 4
Processed Address: میدان سینا, بزرگمهر احمدی, پلاک 4

Original Address: لواسان استخر بوستان جنگلی یاس
Processed Address: لواسان, استخر, بوستان جنگلی, یاس

Original Address: شهرک شهید الماسی بلوار هجرت انتهای هجرت
Processed Address: شهرک شهید الماسی, بلوار هجرت, انتهای هجرت

Original Address: سرخه حصار بزرگراه یاسینی بوستان جنگلی سرخه حصار
Processed Address: Here are the address components separated by commas:

سرخه حصار, بزرگراه یاسینی, بوستان جنگلی, سرخه حصار

Original Address: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب
Processed Address: دماوند, بلوار اتحاد, قبل از اول شرقی, آکس چوب



Code using fine-tuning

In [ ]:
import pandas as pd

# Loading data
df = pd.read_csv('/content/drive/MyDrive/map_dataset.csv')

# Drop unnecessary columns and keep only 'formatted_address'
df = df[['formatted_address']].dropna()

# Create 'Input' column by removing commas from 'formatted_address'
df['Input'] = df['formatted_address'].apply(lambda x: x.replace('،', ''))

# Rename 'formatted_address' to 'Output'
df.rename(columns={'formatted_address': 'Output'}, inplace=True)

# Remove duplicate rows based on the 'Output' column
df.drop_duplicates(subset='Output', inplace=True)

# Print the transformed dataset
print(df.head(50))

                                                 Output  \
1                        امین آباد، اوشان - فشم، ارغوان   
57                         امین آباد، اوشان - فشم، بهار   
107                                 لواسان، اوشان - فشم   
143           لواسان، اوشان - فشم، رستوران شب‌های طهران   
215                                  لواسان، جاده لشکرک   
779                لواسان، جاده لشکرک، بوستان جنگلی یاس   
903                     لواسان، استخر، بوستان جنگلی یاس   
1081  استخر، بوستان جنگلی یاس، آزمایشگاه بتن تحصصی م...   
1277                               تهران، لواسان، استخر   
1311                                       تهران، استخر   
1319                       تهران، استخر، بزرگراه بابایی   
1361                  تهران، استخر، بوستان جنگلی لویزان   
1461                   استخر، مسجد، بوستان جنگلی لویزان   
1519               استخر، سعادتمند، بوستان جنگلی لویزان   
1529        صفایی فراهانی، بختیاری، بوستان جنگلی لویزان   
1551             صفایی فراهانی، بم، بوستان جنگلی لویزان 

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00


Splitting the dataset

In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load the tokenizer and model
model_name = "google/mt5-small"  # A smaller model for simpler hardware requirements
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both train and test datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load the tokenizer and model
model_name = "google/mt5-small"  # A smaller model for simpler hardware requirements
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Char by Char tokenization

In [ ]:
def character_level_tokenize(text):
    """
    Splits a given text into character-level tokens by adding spaces between each character.
    """
    # Add spaces between each character to simulate character-level tokenization
    return " ".join(list(text))

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    inputs = [character_level_tokenize(example) for example in examples['Input']]
    targets = [character_level_tokenize(example) for example in examples['Output']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Set up target labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Assuming datasets is your Hugging Face dataset
tokenized_datasets = datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

In [ ]:
def debug_tokenization(tokenizer, input_text, target_text, max_length=128):
    """
    Function to debug and verify tokenization.

    Args:
    - tokenizer: The tokenizer instance to be used.
    - input_text: The input text to be tokenized.
    - target_text: The target text to be tokenized.
    - max_length: The maximum length for tokenization (padding/truncation).

    Returns:
    - None
    """
    # Apply character-level tokenization
    def character_level_tokenize(text):
        return " ".join(list(text))

    char_level_input = character_level_tokenize(input_text)
    char_level_target = character_level_tokenize(target_text)

    # Tokenize inputs and targets
    tokenized_input = tokenizer(char_level_input, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    tokenized_target = tokenizer(char_level_target, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")

    # Decode token IDs to tokens
    decoded_input_tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'].squeeze().tolist())
    decoded_target_tokens = tokenizer.convert_ids_to_tokens(tokenized_target['input_ids'].squeeze().tolist())

    # Convert token IDs to actual text
    decoded_input_text = tokenizer.decode(tokenized_input['input_ids'].squeeze(), skip_special_tokens=True)
    decoded_target_text = tokenizer.decode(tokenized_target['input_ids'].squeeze(), skip_special_tokens=True)

    # Print original and decoded texts
    print("Original Input Text:", input_text)
    print("Character-Level Tokenized Input Text:", char_level_input)
    print("Decoded Input Tokens:", decoded_input_tokens)
    print("Decoded Input Text:", decoded_input_text)

    print("\nOriginal Target Text:", target_text)
    print("Character-Level Tokenized Target Text:", char_level_target)
    print("Decoded Target Tokens:", decoded_target_tokens)
    print("Decoded Target Text:", decoded_target_text)

    # Print token IDs for comparison
    print("\nTokenized Input IDs:", tokenized_input['input_ids'].squeeze().tolist())
    print("Tokenized Target IDs:", tokenized_target['input_ids'].squeeze().tolist())

# Example usage
input_text = "گلشن احسان (گرمابدری) قربانی قبل از کامکار"
target_text = "گلشن، احسان (گرمابدری)، قربانی قبل از کامکار"

debug_tokenization(tokenizer, input_text, target_text)


Original Input Text: گلشن احسان (گرمابدری) قربانی قبل از کامکار
Character-Level Tokenized Input Text: گ ل ش ن   ا ح س ا ن   ( گ ر م ا ب د ر ی )   ق ر ب ا ن ی   ق ب ل   ا ز   ک ا م ک ا ر
Decoded Input Tokens: ['▁گ', '▁', 'ل', '▁ش', '▁', 'ن', '▁ا', '▁ح', '▁س', '▁ا', '▁', 'ن', '▁(', '▁گ', '▁', 'ر', '▁م', '▁ا', '▁ب', '▁د', '▁', 'ر', '▁', 'ی', '▁', ')', '▁ق', '▁', 'ر', '▁ب', '▁ا', '▁', 'ن', '▁', 'ی', '▁ق', '▁ب', '▁', 'ل', '▁ا', '▁', 'ز', '▁ک', '▁ا', '▁م', '▁ک', '▁ا', '▁', 'ر', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<

Checking

In [ ]:
# Display token ids and their corresponding tokens
target_tokens = tokenizer.convert_ids_to_tokens(target_tokenized['input_ids'])
print("Target Tokens and IDs:")
for token_id, token in zip(target_tokenized['input_ids'], target_tokens):
    print(f"Token ID: {token_id}, Token: {token}")


Target Tokens and IDs:
Token ID: 12176, Token: ▁گل
Token ID: 10457, Token: شن
Token ID: 343, Token: ،
Token ID: 24592, Token: ▁اح
Token ID: 25019, Token: سان
Token ID: 274, Token: ▁(
Token ID: 55694, Token: گرم
Token ID: 4283, Token: اب
Token ID: 31770, Token: دری
Token ID: 29778, Token: )،
Token ID: 10783, Token: ▁قر
Token ID: 27424, Token: بانی
Token ID: 5127, Token: ▁قبل
Token ID: 695, Token: ▁از
Token ID: 8878, Token: ▁کام
Token ID: 7288, Token: کار
Token ID: 1, Token: </s>


Modified code with considering comma as single char

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import re

# Load the tokenizer and model
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Add comma to the tokenizer vocabulary
tokenizer.add_tokens([','])

# Resize the model embeddings to account for the new token
model.resize_token_embeddings(len(tokenizer))

# Custom tokenization function
def custom_tokenize(text):
    # Split the text into tokens, keeping commas separate
    tokens = re.findall(r'\(.*?\)|\S+|,', text)
    return tokens

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']

    model_inputs = tokenizer(
        [' '.join(custom_tokenize(text)) for text in inputs],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [' '.join(custom_tokenize(text)) for text in targets],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Debug function
def debug_tokenization(tokenizer, example_input, example_target):
    # Tokenize the input and target texts
    tokenized_input = tokenizer(' '.join(custom_tokenize(example_input)), return_tensors="pt", truncation=True, padding=True, max_length=128)
    tokenized_target = tokenizer(' '.join(custom_tokenize(example_target)), return_tensors="pt", truncation=True, padding=True, max_length=128, add_special_tokens=False)

    print("Original Input:", example_input)
    print("Original Target:", example_target)

    print("Tokenized Input IDs:", tokenized_input["input_ids"].squeeze().tolist())
    print("Tokenized Target IDs:", tokenized_target["input_ids"].squeeze().tolist())

    decoded_input = tokenizer.decode(tokenized_input["input_ids"].squeeze(), skip_special_tokens=True)
    decoded_target = tokenizer.decode(tokenized_target["input_ids"].squeeze(), skip_special_tokens=True)

    print("Tokenized Input (Decoded):", decoded_input)
    print("Tokenized Target (Decoded):", decoded_target)

# Example usage
example_input = "گلشن احسان (گرمابدری) قربانی قبل از کامکار"
example_target = "گلشن، احسان (گرمابدری)، قربانی قبل از کامکار"

# Run the debug function
debug_tokenization(tokenizer, example_input, example_target)

# Apply tokenization to datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Original Input: گلشن احسان (گرمابدری) قربانی قبل از کامکار
Original Target: گلشن، احسان (گرمابدری)، قربانی قبل از کامکار
Tokenized Input IDs: [12176, 10457, 24592, 25019, 274, 55694, 4283, 31770, 271, 10783, 27424, 5127, 695, 8878, 7288, 1]
Tokenized Target IDs: [12176, 10457, 343, 24592, 25019, 274, 55694, 4283, 31770, 271, 259, 343, 10783, 27424, 5127, 695, 8878, 7288]
Tokenized Input (Decoded): گلشن احسان (گرمابدری) قربانی قبل از کامکار
Tokenized Target (Decoded): گلشن، احسان (گرمابدری) ، قربانی قبل از کامکار


Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/437 [00:00<?, ? examples/s]

Fine-Tuning

T5 small model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load the tokenizer and model
model_name = "t5-small"  # A smaller model for simpler hardware requirements
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both train and test datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision training if your hardware supports it
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

# Save the fine-tuned model
trainer.save_model("fine-tuned-t5-small")
tokenizer.save_pretrained("fine-tuned-t5-small")

# Test on unseen data (optional)
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]  # Example unseen addresses
unseen_inputs = tokenizer(unseen_data, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(unseen_inputs["input_ids"])
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Outputs for Unseen Data:")
for inp, out in zip(unseen_data, decoded_outputs):
    print(f"Input: {inp} -> Predicted Output: {out}")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.047033
2,No log,0.039663
3,0.126000,0.032806


{'eval_loss': 0.032806459814310074, 'eval_runtime': 30.2488, 'eval_samples_per_second': 14.447, 'eval_steps_per_second': 1.818, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted Outputs for Unseen Data:
Input: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب -> Predicted Output:         
Input: نیاوران خیابان باهنر کوچه اقاقیا پلاک 12 -> Predicted Output:       12


MT5 Model

Again tokenization by considering comma as single char (same as the previous code)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import re

# Load the tokenizer and model
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Add comma to the tokenizer vocabulary
tokenizer.add_tokens([','])

# Resize the model embeddings to account for the new token
model.resize_token_embeddings(len(tokenizer))

# Custom tokenization function
def custom_tokenize(text):
    # Split the text into tokens, keeping commas separate
    tokens = re.findall(r'\(.*?\)|\S+|,', text)
    return tokens

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']

    model_inputs = tokenizer(
        [' '.join(custom_tokenize(text)) for text in inputs],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [' '.join(custom_tokenize(text)) for text in targets],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Debug function
def debug_tokenization(tokenizer, example_input, example_target):
    # Tokenize the input and target texts
    tokenized_input = tokenizer(' '.join(custom_tokenize(example_input)), return_tensors="pt", truncation=True, padding=True, max_length=128)
    tokenized_target = tokenizer(' '.join(custom_tokenize(example_target)), return_tensors="pt", truncation=True, padding=True, max_length=128, add_special_tokens=False)

    print("Original Input:", example_input)
    print("Original Target:", example_target)

    print("Tokenized Input IDs:", tokenized_input["input_ids"].squeeze().tolist())
    print("Tokenized Target IDs:", tokenized_target["input_ids"].squeeze().tolist())

    decoded_input = tokenizer.decode(tokenized_input["input_ids"].squeeze(), skip_special_tokens=True)
    decoded_target = tokenizer.decode(tokenized_target["input_ids"].squeeze(), skip_special_tokens=True)

    print("Tokenized Input (Decoded):", decoded_input)
    print("Tokenized Target (Decoded):", decoded_target)
# Example
example_input = "گلشن احسان (گرمابدری) قربانی قبل از کامکار"
example_target = "گلشن، احسان (گرمابدری)، قربانی قبل از کامکار"

# Run the debug function
debug_tokenization(tokenizer, example_input, example_target)

# Apply tokenization to datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyth

Original Input: گلشن احسان (گرمابدری) قربانی قبل از کامکار
Original Target: گلشن، احسان (گرمابدری)، قربانی قبل از کامکار
Tokenized Input IDs: [12176, 10457, 24592, 25019, 274, 55694, 4283, 31770, 271, 10783, 27424, 5127, 695, 8878, 7288, 1]
Tokenized Target IDs: [12176, 10457, 343, 24592, 25019, 274, 55694, 4283, 31770, 271, 259, 343, 10783, 27424, 5127, 695, 8878, 7288]
Tokenized Input (Decoded): گلشن احسان (گرمابدری) قربانی قبل از کامکار
Tokenized Target (Decoded): گلشن، احسان (گرمابدری) ، قربانی قبل از کامکار


Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/437 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments



# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision training if your hardware supports it
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

# Save the fine-tuned model
trainer.save_model("fine-tuned-mt5-smalll")
tokenizer.save_pretrained("fine-tuned-mt5-smalll")

# Test on unseen data (optional)
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]  # Example unseen addresses
unseen_inputs = tokenizer(unseen_data, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(unseen_inputs["input_ids"])
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Outputs for Unseen Data:")
for inp, out in zip(unseen_data, decoded_outputs):
    print(f"Input: {inp} -> Predicted Output: {out}")


Epoch,Training Loss,Validation Loss
1,No log,34.861080


{'eval_loss': 34.861080169677734, 'eval_runtime': 442.8577, 'eval_samples_per_second': 0.987, 'eval_steps_per_second': 0.124, 'epoch': 1.0}


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted Outputs for Unseen Data:
Input: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب -> Predicted Output: <extra_id_0>
Input: نیاوران خیابان باهنر کوچه اقاقیا پلاک 12 -> Predicted Output: <extra_id_0>


Using LORA

In [ ]:
!pip install transformers peft

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import LoraConfig, get_peft_model

# Load the tokenizer and model
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank matrices
    lora_alpha=32,  # LoRA scaling factor
    target_modules=["q", "v"],  # Modules to apply LoRA
    lora_dropout=0.1,  # Dropout rate for LoRA
    bias="none",  # Specify bias handling
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both train and test datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision training if your hardware supports it
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

# Save the fine-tuned model
trainer.save_model("fine-tuned-mt5-small-lora")
tokenizer.save_pretrained("fine-tuned-mt5-small-lora")

# Test on unseen data (optional)
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]
unseen_inputs = tokenizer(unseen_data, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(unseen_inputs["input_ids"])
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Outputs for Unseen Data:")
for inp, out in zip(unseen_data, decoded_outputs):
    print(f"Input: {inp} -> Predicted Output: {out}")


Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/437 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,24.729000,No log


{'eval_runtime': 110.2436, 'eval_samples_per_second': 3.964, 'eval_steps_per_second': 0.499, 'epoch': 3.0}


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted Outputs for Unseen Data:
Input: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب -> Predicted Output: <extra_id_0>
Input: نیاوران خیابان باهنر کوچه اقاقیا پلاک 12 -> Predicted Output: <extra_id_0>


different tokenization

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import re

# Load the tokenizer and model
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Add comma to the tokenizer vocabulary
tokenizer.add_tokens([','])

# Resize the model embeddings to account for the new token
model.resize_token_embeddings(len(tokenizer))

# Custom tokenization function
def custom_tokenize(text):
    # Split the text into tokens, keeping commas separate
    tokens = re.findall(r'\(.*?\)|\S+|,', text)
    return tokens

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']

    model_inputs = tokenizer(
        [' '.join(custom_tokenize(text)) for text in inputs],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [' '.join(custom_tokenize(text)) for text in targets],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Debug function
def debug_tokenization(tokenizer, example_input, example_target):
    # Tokenize the input and target texts
    tokenized_input = tokenizer(' '.join(custom_tokenize(example_input)), return_tensors="pt", truncation=True, padding=True, max_length=128)
    tokenized_target = tokenizer(' '.join(custom_tokenize(example_target)), return_tensors="pt", truncation=True, padding=True, max_length=128, add_special_tokens=False)

    print("Original Input:", example_input)
    print("Original Target:", example_target)

    print("Tokenized Input IDs:", tokenized_input["input_ids"].squeeze().tolist())
    print("Tokenized Target IDs:", tokenized_target["input_ids"].squeeze().tolist())

    decoded_input = tokenizer.decode(tokenized_input["input_ids"].squeeze(), skip_special_tokens=True)
    decoded_target = tokenizer.decode(tokenized_target["input_ids"].squeeze(), skip_special_tokens=True)

    print("Tokenized Input (Decoded):", decoded_input)
    print("Tokenized Target (Decoded):", decoded_target)
# Example
example_input = "گلشن احسان (گرمابدری) قربانی قبل از کامکار"
example_target = "گلشن، احسان (گرمابدری)، قربانی قبل از کامکار"

# Run the debug function
debug_tokenization(tokenizer, example_input, example_target)

# Apply tokenization to datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Original Input: گلشن احسان (گرمابدری) قربانی قبل از کامکار
Original Target: گلشن، احسان (گرمابدری)، قربانی قبل از کامکار
Tokenized Input IDs: [12176, 10457, 24592, 25019, 274, 55694, 4283, 31770, 271, 10783, 27424, 5127, 695, 8878, 7288, 1]
Tokenized Target IDs: [12176, 10457, 343, 24592, 25019, 274, 55694, 4283, 31770, 271, 259, 343, 10783, 27424, 5127, 695, 8878, 7288]
Tokenized Input (Decoded): گلشن احسان (گرمابدری) قربانی قبل از کامکار
Tokenized Target (Decoded): گلشن، احسان (گرمابدری) ، قربانی قبل از کامکار


Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/437 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import LoraConfig, get_peft_model


# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank matrices
    lora_alpha=32,  # LoRA scaling factor
    target_modules=["q", "v"],  # Modules to apply LoRA
    lora_dropout=0.1,  # Dropout rate for LoRA
    bias="none",  # Specify bias handling
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision training if your hardware supports it
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

# Save the fine-tuned model
trainer.save_model("fine-tuned-mt5-small-lora2")
tokenizer.save_pretrained("fine-tuned-mt5-small-lora2")

# Test on unseen data (optional)
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]
unseen_inputs = tokenizer(unseen_data, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(unseen_inputs["input_ids"])
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Outputs for Unseen Data:")
for inp, out in zip(unseen_data, decoded_outputs):
    print(f"Input: {inp} -> Predicted Output: {out}")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,52.826900,No log


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'eval_runtime': 501.9502, 'eval_samples_per_second': 0.871, 'eval_steps_per_second': 0.11, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted Outputs for Unseen Data:
Input: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب -> Predicted Output: <extra_id_0>
Input: نیاوران خیابان باهنر کوچه اقاقیا پلاک 12 -> Predicted Output: <extra_id_0>


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
import re


# Load the tokenizer and model
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Add comma to the tokenizer vocabulary
tokenizer.add_tokens([','])

# Resize the model embeddings to account for the new token
model.resize_token_embeddings(len(tokenizer))

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM  # Specify the task type
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Custom tokenization function
def custom_tokenize(text):
    tokens = re.findall(r'\(.*?\)|\S+|,', text)
    return tokens

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']

    model_inputs = tokenizer(
        [' '.join(custom_tokenize(text)) for text in inputs],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [' '.join(custom_tokenize(text)) for text in targets],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both train and test datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=datasets["train"].column_names)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Enable mixed precision training if GPU is available
    generation_max_length=128,  # Set maximum generation length
    generation_num_beams=4,  # Use beam search for generation
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

# Save the fine-tuned model
trainer.save_model("fine-tuned-mt5-small-lora3")
tokenizer.save_pretrained("fine-tuned-mt5-small-lora3")

# Function to segment address
def segment_address(address):
    inputs = tokenizer(' '.join(custom_tokenize(address)), return_tensors="pt", max_length=128, truncation=True, padding=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test on unseen data
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]

print("Predicted Outputs for Unseen Data:")
for address in unseen_data:
    segmented = segment_address(address)
    print(f"Input: {address}")
    print(f"Segmented: {segmented}")
    print()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/437 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,54.331165
2,No log,49.222553
3,52.693300,48.020676


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'eval_loss': 48.02067565917969, 'eval_runtime': 445.8101, 'eval_samples_per_second': 0.98, 'eval_steps_per_second': 0.123, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Predicted Outputs for Unseen Data:
Input: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب
Segmented: <extra_id_0>

Input: نیاوران خیابان باهنر کوچه اقاقیا پلاک 12
Segmented: <extra_id_0>



In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
import re


# Load the tokenizer and model
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Add comma to the tokenizer vocabulary
tokenizer.add_tokens([','])

# Resize the model embeddings to account for the new token
model.resize_token_embeddings(len(tokenizer))

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM  # Specify the task type
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Custom tokenization function
def custom_tokenize(text):
    tokens = re.findall(r'\(.*?\)|\S+|,', text)
    return tokens

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']

    model_inputs = tokenizer(
        [' '.join(custom_tokenize(text)) for text in inputs],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [' '.join(custom_tokenize(text)) for text in targets],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both train and test datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=datasets["train"].column_names)

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Enable mixed precision training if GPU is available
    generation_max_length=128,  # Set maximum generation length
    generation_num_beams=4,  # Use beam search for generation
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

# Save the fine-tuned model
trainer.save_model("fine-tuned-t5-small")
tokenizer.save_pretrained("fine-tuned-t5-small")

# Function to segment address
def segment_address(address):
    inputs = tokenizer(' '.join(custom_tokenize(address)), return_tensors="pt", max_length=128, truncation=True, padding=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test on unseen data
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]

print("Predicted Outputs for Unseen Data:")
for address in unseen_data:
    segmented = segment_address(address)
    print(f"Input: {address}")
    print(f"Segmented: {segmented}")
    print()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss
1,No log,11.613891
2,No log,3.621704
3,10.930100,0.751415


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'eval_loss': 0.751414954662323, 'eval_runtime': 285.5459, 'eval_samples_per_second': 1.53, 'eval_steps_per_second': 0.193, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Predicted Outputs for Unseen Data:
Input: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب
Segmented:                                                         

Input: نیاوران خیابان باهنر کوچه اقاقیا پلاک 12
Segmented:                                                         



Debug (find out tokenization had problem and solved previously by creating a custom tokenization)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})

# Load the tokenizer and model
model_name = "t5-small"  # A smaller model for simpler hardware requirements
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples['Input']
    targets = examples['Output']

    # Debug: Print a sample of inputs and targets
    print("Sample Input:", inputs[0])
    print("Sample Target:", targets[0])

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding=True)

    # Debug: Print a sample of tokenized inputs
    print("Tokenized Input IDs:", model_inputs["input_ids"][0])

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding=True)

        # Debug: Print a sample of tokenized labels
        print("Tokenized Target IDs:", labels["input_ids"][0])

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both train and test datasets
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Inspect a few tokenized samples to debug
print("Sample tokenized dataset:", tokenized_datasets['train'][0])

# Set up the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision training if your hardware supports it
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

# Save the fine-tuned model
trainer.save_model("fine-tuned-t5-small")
tokenizer.save_pretrained("fine-tuned-t5-small")

# Test on unseen data (optional)
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]  # Example unseen addresses
unseen_inputs = tokenizer(unseen_data, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(unseen_inputs["input_ids"], max_length=50)  # Set max_length to prevent truncation
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Outputs for Unseen Data:")
for inp, out in zip(unseen_data, decoded_outputs):
    print(f"Input: {inp} -> Predicted Output: {out}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

Sample Input: استخر بزرگراه زین الدین مسجد حضرت علی ( ع )
Sample Target: استخر، بزرگراه زین الدین، مسجد حضرت علی ( ع )
Tokenized Input IDs: [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 41, 3, 2, 3, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenized Target IDs: [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 41, 3, 2, 3, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sample tokenized dataset: {'Output': 'صاحب الزمان، نهم غربی تقاطع دانشگاه، دانشکده هوا و فضا', 'Input': 'صاحب الزمان نهم غربی تقاطع دانشگاه دانشکده هوا و فضا', '__index_level_0__': 275199, 'input_ids': [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.209664


Evaluation Results: {'eval_loss': 0.20966391265392303, 'eval_runtime': 46.7978, 'eval_samples_per_second': 9.338, 'eval_steps_per_second': 1.175, 'epoch': 1.0}
Predicted Outputs for Unseen Data:
Input: دماوند بلوار اتحاد قبل از اول شرقی آکس چوب -> Predicted Output: 
Input: نیاوران خیابان باهنر کوچه اقاقیا پلاک 12 -> Predicted Output: 


New Code

In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

# Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,  # The dimension for low-rank matrices (you can adjust this)
    lora_alpha=32,
    lora_dropout=0.1
)

# Apply LoRA to the model
model = get_peft_model(model, peft_config)

# Tokenize your data
def tokenize_function(examples):
    inputs = tokenizer(examples['Input'], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(examples['Output'], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # You can increase the batch size due to lower memory usage
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_total_limit=2,  # Keep only the 2 latest checkpoints
    fp16=True  # Mixed precision to speed up
)

# Initialize trainer with LoRA model
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine-tuned-mt5-lora-model")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset

# Load tokenizer and model (meta-llama or other)
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

# Tokenize your data
def tokenize_function(examples):
    inputs = tokenizer(examples['Input'], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(examples['Output'], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine-tuned-mt5-model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/1747 [00:00<?, ? examples/s]

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Load your fine-tuned model into LangChain
loc_llm = ChatTogether(
    together_api_key=TOGETHER_API_KEY,
    model="./fine-tuned-mt5-model",  # Path to your fine-tuned model
    max_tokens=500,
    temperature=0.1
)


In [ ]:
# Evaluate the model after fine-tuning
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)


In [ ]:
# Test on unseen Persian addresses
unseen_data = ["دماوند بلوار اتحاد قبل از اول شرقی آکس چوب", "نیاوران خیابان باهنر کوچه اقاقیا پلاک 12"]

# Tokenize unseen data
unseen_inputs = tokenizer(unseen_data, return_tensors="pt", padding=True, truncation=True)

# Generate predictions
outputs = model.generate(unseen_inputs["input_ids"])

# Decode the outputs to readable text
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Print the original and segmented outputs
print("Predicted Outputs for Unseen Data:")
for inp, out in zip(unseen_data, decoded_outputs):
    print(f"Original Address: {inp} -> Predicted Segmentation: {out}")
